In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
import xmltodict
from typing import Union # 여러 개의 타입이 허용될 수 있는 상황에서는 typing 모듈의 Union을 사용
from pyspark.sql import SparkSession, functions as F, Row
from pyspark.sql.types import StructField, StructType, DoubleType, IntegerType, TimestampType, StringType

In [2]:
MAX_MEMORY="7g"
spark = SparkSession.builder.appName("covid18")\
                .config("spark.executor.memory", MAX_MEMORY)\
                .config("spark.driver.memory", MAX_MEMORY)\
                .getOrCreate()

22/08/25 11:37:42 WARN Utils: Your hostname, hyunj resolves to a loopback address: 127.0.1.1; using 192.168.0.13 instead (on interface eno1)
22/08/25 11:37:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/25 11:37:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [25]:
class Covid(object):
    def __init__(self):
        self.url='http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
        self.ServiceKey=requests.utils.unquote('Ax%2FnB%2FW10KaDGhIK%2Fds3YS6Jx7V%2BkUR%2FBs8l0L%2Ff6icPTFibZkV1lkgsTvlLWFhI5MhUeRPbqYm5uuFXEovnBA%3D%3D',encoding='utf-8')
        self.start_date=(datetime.now() - timedelta(1)).strftime('%Y%m%d')
        self.end_date=(datetime.now() - timedelta(1)).strftime('%Y%m%d')
    @classmethod
    def get_info(self,start_date: Union[date, datetime], end_date: Union[date,datetime]):
        url = self.url
        params={
            'serviceKey':self.ServiceKey,
            'startCreateDt': self.start_date,
            'endCreateDt': self.end_date,
            'numOfRows': 10,
            'pageNo':1
        }
        get=requests.get(url, params)
        content=get.content
        parsing=xmltodict.parse(content)
        item=parsing['response']['body']['items']['item']
        
        return item
    
    def get_sparkFrame(self, start_date:Union[date, datetime],end_date:Union[date,datetime]):
        convert_method = {
            'accdefrate' : float,
            'accexamcnt' : int,
            'createdt': lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'),
            'deathcnt' : int,
            'seq':int,
            'statedt' : str,
            'statetime' : str,
            'updatedt' : lambda x: datetime.strptime(x, 'Y-%m-%d %H:%M:%S.%f')            
        }
        
    
    
    def get_date(self,url,params):
        url = self.url
        params = self.params
        get=requests.get(url, params)
        content=get.content
        parsing=xmltodict.parse(content)
        item=parsing['response']['body']['items']['item']
        
        return item
        

In [42]:
class Covid:
    def __init__(self):
        self.url='http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
        self.ServiceKey=requests.utils.unquote('Ax%2FnB%2FW10KaDGhIK%2Fds3YS6Jx7V%2BkUR%2FBs8l0L%2Ff6icPTFibZkV1lkgsTvlLWFhI5MhUeRPbqYm5uuFXEovnBA%3D%3D',encoding='utf-8')
        self.start_date=(datetime.now() - timedelta(1)).strftime('%Y%m%d')
        self.end_date=(datetime.now() - timedelta(1)).strftime('%Y%m%d')
    
    def get_info(self,start_date: Union[date, datetime], end_date: Union[date,datetime]):
        url = self.url
        start_date=self.start_date
        end_date=self.end_date
        params={
            'serviceKey':self.ServiceKey,
            'startCreateDt': start_date,
            'endCreateDt': end_date,
            'numOfRows': 10,
            'pageNo':2
        }
        get=requests.get(url, params)
        content=get.content
        parsing=xmltodict.parse(content)
        item=parsing['response']['body']['items']['item']
        
        return item
    
cov = Covid()
start_date=cov.start_date
end_date=cov.end_date
info=cov.get_info(start_date, end_date)
print(info)

{'createDt': '2022-08-24 08:50:29.097', 'deathCnt': '26224', 'decideCnt': '22588550', 'seq': '984', 'stateDt': '20220824', 'stateTime': '00:00', 'updateDt': '2022-08-25 08:52:15.411'}


In [64]:
info

{'createDt': '2022-08-24 08:50:29.097',
 'deathCnt': '26224',
 'decideCnt': '22588550',
 'seq': '984',
 'stateDt': '20220824',
 'stateTime': '00:00',
 'updateDt': '2022-08-25 08:52:15.411'}

In [104]:
columns = ['createDt',
           'deathCnt',
           'decideCnt',
           'seq',
           'stateDt',
           'stateTime',
          'updateDt']

In [105]:
df = spark.createDataFrame(data=value, schema=columns)

TypeError: Can not infer schema for type: <class 'str'>

In [102]:
 for a in info.values():
        print(a)

2022-08-24 08:50:29.097
26224
22588550
984
20220824
00:00
2022-08-25 08:52:15.411


In [75]:
for key,value in info.items():
    print(key,value)

createDt 2022-08-24 08:50:29.097
deathCnt 26224
decideCnt 22588550
seq 984
stateDt 20220824
stateTime 00:00
updateDt 2022-08-25 08:52:15.411


In [82]:
columns=list(info.keys())
print(columns)

['createDt', 'deathCnt', 'decideCnt', 'seq', 'stateDt', 'stateTime', 'updateDt']


In [86]:
value = list(info.values())
print(value)

['2022-08-24 08:50:29.097', '26224', '22588550', '984', '20220824', '00:00', '2022-08-25 08:52:15.411']


In [95]:
item_list = []
for it in item:
    item_dict = {}
    for tag in list(item):
        item_dict[tag.name]=convert_method[tag.name](tag.text)

AttributeError: 'str' object has no attribute 'name'

In [106]:
convert_method ={
    'createDt' : lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'),
    'deathCnt' : int,
    'decideCnt ' : int,
    'seq ' : int,
    'stateDt ' : int,
    'stateTime ':str,
    'updatedt' : lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')
}



df = spark.createDataFrame(data=value, schema=columns)

TypeError: Can not infer schema for type: <class 'str'>

In [59]:
convert_method ={
    'createDt' : lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'),
    'deathCnt' : int,
    'decideCnt ' : int,
    'seq ' : int,
    'stateDt ' : int,
    'stateTime ':str,
    'updatedt' : lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')
}

item_list=[]
item_dict={}
for item in info:
    for  in list(item):
        item_dict[tag.name] = convert_method[tag.name](tag.text)

AttributeError: 'str' object has no attribute 'name'

In [70]:
item_list=[]
for key,value in info.items():
    #df_covid = spark.createDataFrame(key,value, Covid_item_Schema)
    #print(value)
    item_list.append(value)

In [74]:
for item in item_list:
    item_dict={}
    

TypeError: StructType can not accept object '2' in type <class 'str'>

In [73]:
Covid_item_Schema = StructType([StructField('createDt', TimestampType(),True), # 등록일시분초
                                StructField('deathCnt', IntegerType(),True),
                                StructField('decideCnt', IntegerType(),True),
                                StructField('seq', IntegerType(),True),
                                StructField('stateDt', StringType() ,True),
                                StructField('updateDt', TimestampType(),True)])

In [53]:
df_covid = spark.createDataFrame(info, Covid_item_Schema)

TypeError: StructType can not accept object 'createDt' in type <class 'str'>

In [32]:
start_date=(datetime.now() - timedelta(1)).strftime('%Y%m%d')
start_date

'20220823'

In [4]:
url='http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
ServiceKey=requests.utils.unquote('Ax%2FnB%2FW10KaDGhIK%2Fds3YS6Jx7V%2BkUR%2FBs8l0L%2Ff6icPTFibZkV1lkgsTvlLWFhI5MhUeRPbqYm5uuFXEovnBA%3D%3D',encoding='utf-8')

In [10]:
start_date=datetime.now() - timedelta(1)
start_date=start_date.strftime('%Y%m%d')

In [12]:
start_date=datetime.now() - timedelta(2)
start_date=start_date.strftime('%Y%m%d')
end_date=datetime.now() - timedelta(1)
end_date=end_date.strftime('%Y%m%d')

params = {
    'serviceKey':ServiceKey,
    'startCreateDt':start_date,
    'endCreateDt': end_date,
    'numOfRows' : 10,
    'pageNo':1
}

In [13]:
get=requests.get(url, params)

In [14]:
get

<Response [200]>

In [37]:
content=get.content
content
parsing=xmltodict.parse(content)
item=parsing['response']['body']['items']['item']

In [38]:
item

[{'createDt': '2022-08-23 09:00:22.27',
  'deathCnt': '26161',
  'decideCnt': '22449301',
  'seq': '983',
  'stateDt': '20220823',
  'stateTime': '00:00',
  'updateDt': '2022-08-24 08:50:48.44'},
 {'createDt': '2022-08-22 08:10:20.298',
  'deathCnt': '26109',
  'decideCnt': '22299043',
  'seq': '982',
  'stateDt': '20220822',
  'stateTime': '00:00',
  'updateDt': '2022-08-24 08:50:59.733'}]